In [2]:
import tensorflow as tf
from datetime import datetime

In [ ]:
# Models, layers and modules
# learning how to define, save, restore a model

# What is a model?
# function that computes something in tensors (a forward pass)
# Some variable that can be updated

In [5]:
# Defining models and layers in tensorflow
# in tensorflow, most high-level implementations are built on tf.Module class

class SimpleModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)
    self.a_variable = tf.Variable(5.0, name="train_me")
    self.non_trainable_variable = tf.Variable(5.0, trainable=False, name="do_not_train_me")
  def __call__(self, x):
    return self.a_variable * x + self.non_trainable_variable

simple_module = SimpleModule(name="simple")

simple_module(tf.constant(5.0))

print("Trainable variables: " , simple_module.trainable_variables)
print("all variables: ", simple_module.variables)

Trainable variables:  (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>,)
all variables:  (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>, <tf.Variable 'do_not_train_me:0' shape=() dtype=float32, numpy=5.0>)


In [11]:
# two-layer linear layer model made out of modules

# model with two dense layers


# creates random weight and zero biases when initialized when initialized.
# weights and biases are trainable variables.

class Dense(tf.Module):
    def __init__(self, in_features, out_features, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(
            tf.random.normal([in_features, out_features]), name='w'
        )
        self.b = tf.Variable(tf.zeros([out_features], name='b'))
    def __call__ (self, x):
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)

# Module containing two dense layers.
# when called, this module will pass two variables through two dense layers
# and return a result.

class SequentialModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)

        self.dense_1 = Dense(in_features=3, out_features=3)
        self.dense_2 = Dense(in_features=3, out_features=2)

    def __call__(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

my_model = SequentialModule(name = "the_model")

print("Model result : ", my_model(tf.constant([[2.0, 2.0, 2.0]])))

print("Submodules : ", my_model.submodules)


for var in my_model.variables:
    print(var, "\n")


Model result :  tf.Tensor([[0.        0.5357013]], shape=(1, 2), dtype=float32)
Submodules :  (<__main__.Dense object at 0x000001AAF36F84C8>, <__main__.Dense object at 0x000001AAF36F8148>)
<tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 3) dtype=float32, numpy=
array([[-0.29807675, -0.23696312,  0.4931119 ],
       [-0.1464643 , -0.260403  , -1.0790645 ],
       [-0.35388803,  0.7964112 , -0.6294454 ]], dtype=float32)> 

<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
array([[ 2.6307747 ,  1.3845483 ],
       [-0.50820696,  0.89568645],
       [ 0.5427004 , -2.25171   ]], dtype=float32)> 



In [13]:
# Waiting to create variables
# with deferring variable creation to first moduel call,
# we don't have to pre-specify variable dimension.

# this is why we need to specify only the output dimesion in keras.Dense layers

class FlexibleDenseModule(tf.Module):
    def __init__(self, out_features, name=None):
        super().__init__(name=name)
        self.is_built = False
        self.out_features = out_features

    def __call__(self, x):
        if not self.is_built:
            self.w = tf.Variable(
                tf.random.normal([x.shape[-1], self.out_features]), name='w'
            )
            self.b = tf.Variable(tf.zeros([self.out_features]), name = 'b')
            self.is_built = True
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)

class MySequentialModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)

        self.dense_1 = FlexibleDenseModule(out_features=3)
        self.dense_2 = FlexibleDenseModule(out_features=2)

    def __call__(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)


my_model = MySequentialModule(name = "the_model")
print("Module result :", my_model(tf.constant([[2.0,2.0,2.0]])))

Module result : tf.Tensor([[0.       3.471935]], shape=(1, 2), dtype=float32)


In [15]:
# saving weights with Checkpoints / SavedModel

# Checkpoints are only weights in the model
# this will create file in current directory

chkp_path = "my_checkpoint"
checkpoint = tf.train.Checkpoint(model = my_model)
checkpoint.write(chkp_path)

# in this directory,
# my_checkpoint.data-00000-of-000000 and
# my_checkpoint.index
# files are created.

# to look inside a checkpoint
tf.train.list_variables(chkp_path)

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('model/dense_1/b/.ATTRIBUTES/VARIABLE_VALUE', [3]),
 ('model/dense_1/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 3]),
 ('model/dense_2/b/.ATTRIBUTES/VARIABLE_VALUE', [2]),
 ('model/dense_2/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 2])]

In [16]:
# you can load model with checkpoint

new_model = MySequentialModule()
new_checkpoint = tf.train.Checkpoint(model=new_model)
new_checkpoint.restore("my_checkpoint")

new_model(tf.constant([[2.0,2.0,2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.      , 3.471935]], dtype=float32)>